In [ ]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from keras.utils import np_utils
import matplotlib.pyplot as plt

# 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
tf.random.set_seed(777)

#######################################################################################################################
# data 불러오기
xy = pd.read_csv('New_DataSet.csv',encoding='CP949')
xy.info()

x = xy.values[:,0:4].astype(np.float)
y = xy.values[:,6].astype(np.str)

# 하이퍼 파라미터
timesteps = seq_length = 27
epochs = 300

# Labeling -> Y data
for i in range (len(y)):
   if y[i] == '1 0 0 0 ':
      y[i] = 0
   elif y[i] == '0 1 0 0':
      y[i] = 1
   elif y[i] == '0 0 1 0':
      y[i] = 2
   # elif y[i] == '0 0 0 1':
   #    y[i] = 3

dataX = [] # 입력으로 사용될 Sequence Data
dataY = [] # 출력

# Data Set
for i in range(0, int(len(x)/seq_length)):
   _x = x[(i*27):(i*27) + seq_length]
   dataX.append(_x)
   _y = y[(i*27) + seq_length-1]
   dataY.append(_y)

# dataSet shuffle
tmp = [[x,y] for x, y in zip(dataX, dataY)]
random.shuffle(tmp)

# After suffle, seperate data X, Y
dataX = [n[0] for n in tmp]
dataY = [n[1] for n in tmp]

# 학습용/테스트용 data 생성
train_size = int(len(tmp) * 0.9)
test_size = len(tmp) - train_size

print(dataX[0].shape)

# data를 잘라 train data 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])
trainY = np_utils.to_categorical(np.reshape(trainY, (len(trainY), 1)))
# trainY = np.reshape(trainY, (len(trainY), 1))

# data를 잘라 test data 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])
testY = np_utils.to_categorical(np.reshape(testY, (len(testY), 1)))
# testY = np.reshape(testY, (len(testY), 1))
print(testX[0])


# TensorFlow 2.0########################################################################################################
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Bidirectional(LSTM(128,
                             dropout=0.25,
                             recurrent_dropout=0.25,
                             return_sequences=True),input_shape=(27,4)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(256,
                             dropout=0.25,
                             recurrent_dropout=0.25,
                             return_sequences=False)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer=Adam(lr=0.001),
              loss='mse',
              metrics=['accuracy'])

history = model.fit(trainX,
                    trainY,
                    epochs=epochs,
                    batch_size=128,
                    # validation_split=0.2,
                    validation_data=(testX, testY),
                    verbose=2)

results = model.evaluate(testX,  testY, verbose=0)
print(results)

history_dict = history.history
history_dict.keys()

# Train loss
plt.plot(history_dict['loss'], label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.legend()
plt.show()

# Train Acc
plt.plot(history_dict['accuracy'], label='Train Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train Accuracy')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.legend()
plt.show()

# Val loss
# plt.plot(history_dict['val_loss'], label='Loss(LSTM)', color='yellow')
plt.plot(history_dict['val_loss'],color='b')
plt.xlabel('Epoch')
plt.ylabel('Loss')
# plt.title('Validation Loss')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.legend(loc='upper left')
plt.show()

# val acc
# plt.plot(history_dict['val_accuracy'], label='Accuracy(LSTM)', color='yellow')
plt.plot(history_dict['val_accuracy'],color='b')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.title('Validation Accuracy')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.legend(loc='upper left')
plt.show()